In [ ]:
epochs = 10

my_timer  = Timer()
for epoch in range(epochs):

    print("dfghs\n")
    # my_timer.start()
    for id,(data,target) in enumerate(train_loader):

        if id == 0:
            # print(my_timer.stop())
            print("dfghs\n")

print('avg_time:{}'.format(
            my_timer.avg()          
        ))
